# Task 3 - Model Explainability

In [2]:
# %pip install shap

In [4]:
import pandas as pd
import numpy as np
import shap
import os
import sys
import ipaddress
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
sys.path.append(os.path.abspath("../Script"))
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, average_precision_score, confusion_matrix, classification_report
import warnings
warnings.filterwarnings("ignore")

In [5]:
os.chdir("..")
# print(os.getcwd()) 

In [11]:
from data_load import *
from model_explain import *

In [7]:
    # Load datasets
fraud_df = load_and_clean_data("Data/Fraud_Data.csv")
ip_df = load_and_clean_data("Data/IpAddress_to_Country.csv")
credit_df = load_and_clean_data("Data/creditcard.csv")

Loaded: Data/Fraud_Data.csv | Rows: 151112, Columns: 11
Cleaned: Removed duplicates and empty rows.
Loaded: Data/IpAddress_to_Country.csv | Rows: 138846, Columns: 3
Cleaned: Removed duplicates and empty rows.
Loaded: Data/creditcard.csv | Rows: 284807, Columns: 31
Cleaned: Removed duplicates and empty rows.


In [ ]:
X_train_bal_scaled (numpy or pd.DataFrame) after SMOTE + scaling
xgb_model (trained XGBoostClassifier)

# Convert back to DataFrame if needed (important for feature names in SHAP)
if not isinstance(X_train_bal_scaled, pd.DataFrame):
    X_train_df = pd.DataFrame(X_train_bal_scaled, columns=X.columns)
else:
    X_train_df = X_train_bal_scaled

# Interpret model
explain_model_with_shap(model=xgb_model, X_train=X_train_df, X_sample=0)
Call SHAP interpretation function
explain_model_with_shap(best_model, X_train_scaled, X_sample=5)